In [1]:
import pandas as pd 
import numpy as np 

data = pd.read_csv('new_data/merge_data.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581398 entries, 0 to 581397
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Company  581398 non-null  object 
 1   Date     581398 non-null  object 
 2   Close    581398 non-null  float64
 3   Volume   581398 non-null  int64  
 4   CPI      581398 non-null  float64
 5   USTS     581398 non-null  float64
 6   DI       581398 non-null  float64
 7   PP       581398 non-null  float64
 8   IR       581398 non-null  float64
 9   OP       581398 non-null  float64
 10  CLF      581398 non-null  int64  
 11  EPU      581398 non-null  float64
 12  GDP      581398 non-null  float64
 13  UNEMP    581398 non-null  float64
dtypes: float64(10), int64(2), object(2)
memory usage: 62.1+ MB


In [3]:
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
# 종목별 상관관계가 가장 높은 변수 확인

company_list = []
index_list = []

for i in data.Company.unique():
    data_colse = data[data['Company'] == i].corr()['Close']
    max_index = data_colse.abs().nlargest(2).iloc[1:].index[0]
    index_list.append(max_index)
    company_list.append(i)

len(index_list)
len(company_list)

C:\Users\yju12\AppData\Local\Temp\ipykernel_13528\2453781572.py:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_colse = data[data['Company'] == i].corr()['Close']
C:\Users\yju12\AppData\Local\Temp\ipykernel_13528\2453781572.py:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_colse = data[data['Company'] == i].corr()['Close']
C:\Users\yju12\AppData\Local\Temp\ipykernel_13528\2453781572.py:7: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_colse = data[data['Compan

491

In [5]:
company_corr = pd.DataFrame({'Comapny':company_list,
                             'max_corr':index_list})

company_corr

,Comapny,max_corr
0,A,GDP
1,AAPL,CPI
2,ABBV,CPI
3,ABEV,UNEMP
4,ABNB,DI
...,...,...
486,YUM,GDP
487,ZBH,DI
488,ZM,PP
489,ZS,GDP


In [6]:
# 대체적으로 GDP, CPI(소비자물가지수)가 대부분 주식종목의 가격과 상관성이 높음
# OP(원유가격), DI(달러지수)에 가장 큰 영향을 받는 종목도 존재..  
company_corr.groupby('max_corr').size()

max_corr
CLF         6
CPI       206
DI         36
EPU         6
GDP       116
IR         22
OP         40
PP         25
UNEMP       7
USTS       24
Volume      3
dtype: int64

In [7]:
# 노동인구수와 가장 영향이 높은 종목
company_corr[company_corr['max_corr'] == 'OP']['Comapny']

40      AVB
48      BAC
51      BCE
58     BIIB
61      BKR
63      BMO
65       BN
66      BNS
94       CM
96      CME
122     CVS
129    DDOG
142     DTE
158     ENB
163     EQR
170     EXR
182    FITB
191     GFS
203     GSK
217     HPQ
253     KMI
258     LHX
267     LVS
270     LYV
289      MO
301     MTB
317     NTR
326       O
343     PFE
352     PNC
355     PRU
366     RCI
377     RTX
385    SCHW
414     SYY
418      TD
426     TLK
433     TRP
443      TU
473    WELL
Name: Comapny, dtype: object

---

# pycaret ML

In [8]:
from pycaret.time_series import TSForecastingExperiment

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581398 entries, 0 to 581397
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   Company  581398 non-null  object        
 1   Date     581398 non-null  datetime64[ns]
 2   Close    581398 non-null  float64       
 3   Volume   581398 non-null  int64         
 4   CPI      581398 non-null  float64       
 5   USTS     581398 non-null  float64       
 6   DI       581398 non-null  float64       
 7   PP       581398 non-null  float64       
 8   IR       581398 non-null  float64       
 9   OP       581398 non-null  float64       
 10  CLF      581398 non-null  int64         
 11  EPU      581398 non-null  float64       
 12  GDP      581398 non-null  float64       
 13  UNEMP    581398 non-null  float64       
dtypes: datetime64[ns](1), float64(10), int64(2), object(1)
memory usage: 62.1+ MB


In [10]:
# 애플 주가예측
AAPL = data[data['Company'] == 'AAPL']
AAPL

,Company,Date,Close,Volume,CPI,USTS,DI,PP,IR,OP,CLF,EPU,GDP,UNEMP
1214,AAPL,2018-11-29,43.083508,167080000,252.594,3.03,116.1990,124267000.0,2.20,58.29,104047,183.21,61619.0,3.8
1215,AAPL,2018-11-30,42.850754,158126000,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047,136.43,61619.0,3.8
1216,AAPL,2018-12-03,44.348064,163210000,252.767,2.98,115.9081,124163000.0,2.19,60.17,104131,158.49,61619.0,3.9
1217,AAPL,2018-12-04,42.397247,165377200,252.767,2.91,116.0051,124163000.0,2.20,61.22,104131,109.29,61619.0,3.9
1218,AAPL,2018-12-06,41.924541,172393600,252.767,2.87,116.4019,124163000.0,2.20,57.83,104131,110.19,61619.0,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,AAPL,2023-09-25,175.848328,46172700,307.481,4.55,122.5803,129115000.0,5.33,94.01,106975,14.69,67083.0,3.8
2424,AAPL,2023-09-26,171.733749,64588900,307.481,4.56,122.9224,129115000.0,5.33,94.46,106975,143.47,67083.0,3.8
2425,AAPL,2023-09-27,170.205750,66921800,307.481,4.61,123.4153,129115000.0,5.33,97.10,106975,150.04,67083.0,3.8
2426,AAPL,2023-09-28,170.465424,56294400,307.481,4.59,122.9827,129115000.0,5.33,96.64,106975,111.89,67083.0,3.8


In [11]:
AAPL.set_index('Date', inplace = True)

In [12]:
AAPL.index = AAPL.index.to_period(freq = 'd')

In [13]:
AAPL.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 1214 entries, 2018-11-29 to 2023-09-29
Freq: D
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Company  1214 non-null   object 
 1   Close    1214 non-null   float64
 2   Volume   1214 non-null   int64  
 3   CPI      1214 non-null   float64
 4   USTS     1214 non-null   float64
 5   DI       1214 non-null   float64
 6   PP       1214 non-null   float64
 7   IR       1214 non-null   float64
 8   OP       1214 non-null   float64
 9   CLF      1214 non-null   int64  
 10  EPU      1214 non-null   float64
 11  GDP      1214 non-null   float64
 12  UNEMP    1214 non-null   float64
dtypes: float64(10), int64(2), object(1)
memory usage: 132.8+ KB


In [14]:
# EDA 실험 만들기
eda = TSForecastingExperiment()

In [15]:
# # 주말, 공휴일의 경우 datetime이 비기 때문에 모든 날짜를 생성 할 필요가 있음..
# eda.setup(data=AAPL, target = 'Close', fh=48, session_id=1)

- 주말 및 공휴일 날짜 생성

In [16]:
# 주말, 공휴일의 대체값 생성
AAPL.reset_index(inplace = True)
AAPL

,Date,Company,Close,Volume,CPI,USTS,DI,PP,IR,OP,CLF,EPU,GDP,UNEMP
0,2018-11-29,AAPL,43.083508,167080000,252.594,3.03,116.1990,124267000.0,2.20,58.29,104047,183.21,61619.0,3.8
1,2018-11-30,AAPL,42.850754,158126000,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047,136.43,61619.0,3.8
2,2018-12-03,AAPL,44.348064,163210000,252.767,2.98,115.9081,124163000.0,2.19,60.17,104131,158.49,61619.0,3.9
3,2018-12-04,AAPL,42.397247,165377200,252.767,2.91,116.0051,124163000.0,2.20,61.22,104131,109.29,61619.0,3.9
4,2018-12-06,AAPL,41.924541,172393600,252.767,2.87,116.4019,124163000.0,2.20,57.83,104131,110.19,61619.0,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,2023-09-25,AAPL,175.848328,46172700,307.481,4.55,122.5803,129115000.0,5.33,94.01,106975,14.69,67083.0,3.8
1210,2023-09-26,AAPL,171.733749,64588900,307.481,4.56,122.9224,129115000.0,5.33,94.46,106975,143.47,67083.0,3.8
1211,2023-09-27,AAPL,170.205750,66921800,307.481,4.61,123.4153,129115000.0,5.33,97.10,106975,150.04,67083.0,3.8
1212,2023-09-28,AAPL,170.465424,56294400,307.481,4.59,122.9827,129115000.0,5.33,96.64,106975,111.89,67083.0,3.8


In [17]:
# 주말, 공휴일의 날짜 비어있는 데이터 생성
from pycaret.utils.time_series import clean_time_index

cleaned_data = clean_time_index(data = AAPL, index_col = 'Date', freq = 'd')
cleaned_data

,Company,Close,Volume,CPI,USTS,DI,PP,IR,OP,CLF,EPU,GDP,UNEMP
2018-11-29,AAPL,43.083508,167080000.0,252.594,3.03,116.1990,124267000.0,2.20,58.29,104047.0,183.21,61619.0,3.8
2018-11-30,AAPL,42.850754,158126000.0,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047.0,136.43,61619.0,3.8
2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-03,AAPL,44.348064,163210000.0,252.767,2.98,115.9081,124163000.0,2.19,60.17,104131.0,158.49,61619.0,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,AAPL,175.848328,46172700.0,307.481,4.55,122.5803,129115000.0,5.33,94.01,106975.0,14.69,67083.0,3.8
2023-09-26,AAPL,171.733749,64588900.0,307.481,4.56,122.9224,129115000.0,5.33,94.46,106975.0,143.47,67083.0,3.8
2023-09-27,AAPL,170.205750,66921800.0,307.481,4.61,123.4153,129115000.0,5.33,97.10,106975.0,150.04,67083.0,3.8
2023-09-28,AAPL,170.465424,56294400.0,307.481,4.59,122.9827,129115000.0,5.33,96.64,106975.0,111.89,67083.0,3.8


In [18]:
cleaned_data = cleaned_data.fillna(method='ffill')
cleaned_data

,Company,Close,Volume,CPI,USTS,DI,PP,IR,OP,CLF,EPU,GDP,UNEMP
2018-11-29,AAPL,43.083508,167080000.0,252.594,3.03,116.1990,124267000.0,2.20,58.29,104047.0,183.21,61619.0,3.8
2018-11-30,AAPL,42.850754,158126000.0,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047.0,136.43,61619.0,3.8
2018-12-01,AAPL,42.850754,158126000.0,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047.0,136.43,61619.0,3.8
2018-12-02,AAPL,42.850754,158126000.0,252.594,3.01,116.4157,124267000.0,2.20,57.71,104047.0,136.43,61619.0,3.8
2018-12-03,AAPL,44.348064,163210000.0,252.767,2.98,115.9081,124163000.0,2.19,60.17,104131.0,158.49,61619.0,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25,AAPL,175.848328,46172700.0,307.481,4.55,122.5803,129115000.0,5.33,94.01,106975.0,14.69,67083.0,3.8
2023-09-26,AAPL,171.733749,64588900.0,307.481,4.56,122.9224,129115000.0,5.33,94.46,106975.0,143.47,67083.0,3.8
2023-09-27,AAPL,170.205750,66921800.0,307.481,4.61,123.4153,129115000.0,5.33,97.10,106975.0,150.04,67083.0,3.8
2023-09-28,AAPL,170.465424,56294400.0,307.481,4.59,122.9827,129115000.0,5.33,96.64,106975.0,111.89,67083.0,3.8


In [19]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 1766 entries, 2018-11-29 to 2023-09-29
Freq: D
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Company  1766 non-null   object 
 1   Close    1766 non-null   float64
 2   Volume   1766 non-null   float64
 3   CPI      1766 non-null   float64
 4   USTS     1766 non-null   float64
 5   DI       1766 non-null   float64
 6   PP       1766 non-null   float64
 7   IR       1766 non-null   float64
 8   OP       1766 non-null   float64
 9   CLF      1766 non-null   float64
 10  EPU      1766 non-null   float64
 11  GDP      1766 non-null   float64
 12  UNEMP    1766 non-null   float64
dtypes: float64(12), object(1)
memory usage: 193.2+ KB


In [20]:
cleaned_data.drop(columns = 'Company', inplace = True)

In [21]:
eda.setup(data = cleaned_data, 
          target = 'Close', 
          fh=48, 
          session_id=1)

,Description,Value
0,session_id,1
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(1766, 12)"
5,Transformed data shape,"(1766, 12)"
6,Transformed train set shape,"(1718, 12)"
7,Transformed test set shape,"(48, 12)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [1]:
# 시계열 데이터 시각화
eda.plot_model(
    fig_kwargs={
        'renderer':'notebook',
        'width':1000,
        'height':1200,
    }
)

NameError: name 'eda' is not defined